In [1]:
import matplotlib.pyplot as plt
import mpl_toolkits
import numpy as np
from matplotlib import cm
from numpy.typing import NDArray


# Define the surface function f(x, y)
def draw_surface(
    x_limits: NDArray[np.float64],
    y_limits: NDArray[np.float64],
    surface_fn: callable,
) -> mpl_toolkits.mplot3d.axes3d.Axes3D:
    X, Y = np.meshgrid(x_limits, x_limits)
    Z = surface_fn((X, Y))
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection="3d")

    # Plot the loss surface
    ax.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.6, edgecolor="none")
    return ax


import copy


def saddle(params):
    x, y = params
    return x**2 - y**2


def camel(params):
    x, y = params
    return 2 * x**2 - 1.05 * x**4 + (x**6) / 6 + x * y + y**2


def numerical_gradient(f: callable, params, h=1e-4):
    results = np.zeros_like(params)
    for i in range(len(params)):
        original_val = params[i]
        params[i] = original_val + h
        f_plus = f(params)
        params[i] = original_val - h
        f_minus = f(params)
        results[i] = (f_plus - f_minus) / (2 * h)
        params[i] = original_val
    return results

In [2]:
def plane(params):
    x, y = params
    return x + 0.5 * y


def calculate_trajectory(
    x_start,
    y_start,
    surface_fn,
    x_limits: NDArray[np.float64] = np.linspace(-2, 2, 100),
    y_limits: NDArray[np.float64] = np.linspace(-2, 2, 100),
    g: NDArray[np.float64] = np.array([0, 0, -9.8]),
    iterations=5,
    lr=0.1,
    k=1,
):  # documenation for calculate_trajectory
    """
    Calculate the trajectory of an object on a given surface.

    Parameters:
    x_start (float): Initial x position.
    y_start (float): Initial y position.
    surface_fn (callable): Function representing the surface.
    x_limits (NDArray[np.float64]): Range of x values (default: np.linspace(-2, 2, 100)).
    y_limits (NDArray[np.float64]): Range of y values (default: np.linspace(-2, 2, 100)).
    g (NDArray[np.float64]): Gravitational force vector (default: np.array([0, 0, -9.8])).
    iterations (int): Number of iterations to calculate the trajectory (default: 5).
    lr (float): Learning rate for position updates (default: 0.1).
    k (float): Friction coefficient (default: 1).

    Returns:
    trajectory (list): List of positions representing the trajectory.
    """
    starting_xy = np.array([x_start, y_start])
    start = np.array([*starting_xy, surface_fn(starting_xy)])
    norm = np.array([*numerical_gradient(surface_fn, starting_xy), -1])
    norm = norm / np.linalg.norm(norm)
    g_norm = np.dot(g, norm) * norm
    g_tangent = g - g_norm

    current_pos = start
    a = g_tangent
    trajectory = []
    trajectory.append(current_pos)
    v = np.array([0, 0, 0])
    for i in range(iterations):
        v = v + a * lr
        norm = np.array(
            [*numerical_gradient(surface_fn, current_pos[:2]), -1]
        )
        norm = norm / np.linalg.norm(norm)
        v_norm = np.dot(v, norm) * norm
        v = v - v_norm
        next_pos = current_pos + v * lr
        g_norm = np.dot(g, norm) * norm
        friction = np.linalg.norm(g_norm) * k
        friction_force = -v * friction
        g_tangent = g - g_norm
        a = g_tangent + friction_force
        next_pos[2] = surface_fn(next_pos[:2])

        current_pos = next_pos
        trajectory.append(current_pos)
        # ax.plot(*current_pos, "ro")
        # Plot current_pos on ax

    # plt.show()
    return np.array(trajectory)


trajectory = calculate_trajectory(
    x_start=1.0, y_start=1.0, surface_fn=camel
)
trajectory

array([[1.        , 1.        , 3.11666667],
       [0.98667674, 0.97779456, 3.02653999],
       [0.96338867, 0.9389937 , 2.87133503],
       [0.9320953 , 0.88768273, 2.66972931],
       [0.89392426, 0.82703081, 2.43604004],
       [0.8494904 , 0.75962652, 2.18143494]])

In [3]:
from ipywidgets import AppLayout, FloatSlider

In [4]:
%matplotlib ipympl


def draw_surface(
    x_limits: NDArray[np.float64],
    y_limits: NDArray[np.float64],
    surface_fn: callable,
) -> mpl_toolkits.mplot3d.axes3d.Axes3D:
    X, Y = np.meshgrid(x_limits, x_limits)
    Z = surface_fn((X, Y))
    fig = plt.figure()
    fig.canvas.header_visible = False

    # fig.canvas.layout.min_height = "400px"
    trajectory = calculate_trajectory(
        x_start=1.0, y_start=1.0, surface_fn=camel, iterations=200
    )
    trajectory = np.array(trajectory)
    ax = fig.add_subplot(111, projection="3d")
    # Plot the loss surface
    ax.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.6, edgecolor="none")
    points = ax.plot(*[trajectory[:, i] for i in range(3)])
    # plt.show()
    return fig, points


plt.ioff()

fig, points = draw_surface(
    x_limits=np.linspace(-2, 2, 100),
    y_limits=np.linspace(-2, 2, 100),
    surface_fn=camel,
)

slider = FloatSlider(
    orientation="horizontal",
    description="Friction:",
    value=1.0,
    min=0.02,
    max=2.0,
)

slider.layout.margin = "0px 30% 0px 30%"
slider.layout.width = "40%"


def update_points(change):
    new_trajectory = calculate_trajectory(
        x_start=1.0,
        y_start=1.0,
        surface_fn=camel,
        iterations=200,
        k=change.new,
    )
    points[0].set_data(new_trajectory[:, 0], new_trajectory[:, 1])
    points[0].set_3d_properties(new_trajectory[:, 2])
    fig.canvas.draw()
    fig.canvas.flush_events()


slider.observe(update_points, names="value")
AppLayout(center=fig.canvas, footer=slider, pane_heights=[0, 6, 1])

AppLayout(children=(FloatSlider(value=1.0, description='Friction:', layout=Layout(grid_area='footer', margin='…